#### Имеется одноканальная СМО с отказами. Поток заявок — простейший с интенсивностью  X. Время обслуживания — не случайное и в точности равно to6c= 1 / X. Найти относительную и пропускную способность СМО в предельном стационарном режиме. 

In [5]:
import random


def simulate_queuing_system(lambd: float, my: float, simulation_time: int) -> tuple:
    total_requests: int = 0
    serviced_requests: int = 0
    rejected_requests: int = 0
    service_time: float = 1 / service_rate
    time: float = 0

    while time < simulation_time:
        interarrival_time = random.expovariate(arrival_rate)
        total_requests += 1

        if interarrival_time < service_time:
            time += interarrival_time
            rejected_requests += 1
        else:
            time += service_time
            serviced_requests += 1

    return total_requests, serviced_requests, rejected_requests


# Параметры модели
arrival_rate = float(input('Введите интенсивность поступления заявок: '))  # Интенсивность поступления заявок (λ)
service_rate = float(input('Введите интенсивность обслуживания: '))  # Интенсивность обслуживания (X)
simulation_time = 10 ** 6

# Теоретические расчеты
ro = arrival_rate / service_rate
p1 = ro / (1 + ro) # Вероятность отказа
p0 = 1 / (1 + ro) # Вероятность обслуживания 
theoretical_relative_throughput = p0  # Теоретическая относительная пропускная способность
theoretical_absolute_throughput = arrival_rate * theoretical_relative_throughput  # Теоретическая абсолютная пропускная способность

# Запускаем симуляцию
total_arrivals, serviced_requests, rejected_requests = simulate_queuing_system(lambd=arrival_rate, my=service_rate,
                                                                               simulation_time=simulation_time)

# Практические расчеты
practical_relative_throughput = serviced_requests / total_arrivals
practical_absolute_throughput = arrival_rate * practical_relative_throughput

# Выводим результаты
print("Теоретические расчеты:")
print("Относительная пропускная способность (R): {:.2f}".format(theoretical_relative_throughput))
print("Абсолютная пропускная способность (A): {:.2f}".format(theoretical_absolute_throughput))
print("\nПрактические расчеты:")
print("Относительная пропускная способность (R): {:.2f}".format(practical_relative_throughput))
print("Абсолютная пропускная способность (A): {:.2f}".format(practical_absolute_throughput))


Теоретические расчеты:
Относительная пропускная способность (R): 0.67
Абсолютная пропускная способность (A): 3.33

Практические расчеты:
Относительная пропускная способность (R): 0.61
Абсолютная пропускная способность (A): 3.03


#### Железнодорожная сортировочная горка, на которую подается простейший поток составов с интенсивностью X = 2 состава в час, представляет собой одноканальную СМО с неограниченной очередью. Время обслуживания (роспуска) состава на горке имеет показательное распределение со средним значением tобсл = 20 мин. Найти финальные вероятности состояний СМО, среднее число z составов, связанных с горкой, среднее число составов в очереди, среднее время tCOCT пребывания состава в СМО, среднее время tQ пребывания состава в очереди.

In [2]:
l = 2 / 60
t_ob = 20

ro = l * t_ob
q = 1
A = q * l
l_queue = ro ** 2 / (1 - ro)
t_queue = l_queue / A
l_ob = ro
l_sys = l_ob + l_queue
t_sys = l_sys / A

print(f"Интенсивнось {l}")
print(f"Время обслуживания {t_ob} мин")
print(f"Нагрузка {ro}")
print(f"Вероятность простоя {1 - ro}")
print(f"Число составов связанных с горкой {l_sys}")
print(f"Число составов в очереди {l_queue}")
print(f"Время в смо {t_sys} мин")
print(f"Время в очереди {t_queue} мин")

Интенсивнось 0.03333333333333333
Время обслуживания 20 мин
Нагрузка 0.6666666666666666
Вероятность простоя 0.33333333333333337
Число составов связанных с горкой 1.9999999999999996
Число составов в очереди 1.333333333333333
Время в смо 59.999999999999986 мин
Время в очереди 39.99999999999999 мин


In [3]:
import numpy as np

np.random.seed(542)

In [4]:
class Request:
    def __init__(self, current_time):
        self.queue_time = current_time


class Stats:
    def __init__(self, duration):
        self.duration = duration
        self.process_time_deltas = []
        self.queue_time_deltas = []
        self.system_time_deltas = []

    def get_stats(self):
        system_load = sum(self.process_time_deltas) / self.duration
        queue_size = system_load ** 2 / (1 - system_load)
        res = {
            "system_load": system_load,
            "downtime_probability": 1 - system_load,
            "process_time": sum(self.process_time_deltas),
            "downtime": self.duration - sum(self.process_time_deltas),
            "avg_queue_size": queue_size,
            "avg_system_size": queue_size + system_load,
            "queue_time": queue_size / A,
            "system_time": (queue_size + system_load) / A
        }
        return res


class Service:
    def __init__(self, duration, arr_time, serve_time):
        self.duration = duration
        self.serve_time = serve_time
        self.arr_time = arr_time
        self.current_time = 0
        self.requests = []
        self.current_processing_request = []
        self.request_timeline = self.generate_timeline(self.arr_time)
        self.stats = Stats(duration=self.duration)

    def generate_timeline(self, time):
        i = 1
        res = []
        while i * time < self.duration:
            res.append(i * time)
            i += 1
        return res

    def get_request_time_delta(self):
        return np.random.exponential(self.serve_time)

    def reset_service(self):
        self.current_time = 0
        self.requests = []

    def add_new_requests(self, time_delta):
        type_a = list(
            filter(lambda x: self.current_time <= x < self.current_time + time_delta, self.request_timeline))

        for _ in range(len(type_a)):
            self.requests.append(Request(current_time=self.current_time))

    def process_request(self, delta):
        if len(self.current_processing_request) == 1 and self.current_time + delta >= \
                self.current_processing_request[0][1]:
            request, time = self.current_processing_request.pop(0)
            self.stats.system_time_deltas.append(time - request.queue_time)
        if len(self.requests) > 0 and len(self.current_processing_request) == 0:
            request: Request = self.requests.pop(0)
            process_time_delta = self.get_request_time_delta()
            self.stats.queue_time_deltas.append(self.current_time - request.queue_time)
            self.stats.process_time_deltas.append(process_time_delta)
            self.current_processing_request.append((request, self.current_time + process_time_delta))

    def run(self):
        delta = 1
        self.reset_service()
        while self.current_time < self.duration:
            self.process_request(delta)
            self.add_new_requests(delta)
            self.current_time += delta


DURATION = 20000
service = Service(
    duration=DURATION,
    arr_time=1 / l,
    serve_time=t_ob
)
service.run()
stats = service.stats.get_stats()

print(f"Нагрузка системы: {stats['system_load']}")
print(f"Вероятность простоя системы: {stats['downtime_probability']}")
print(f"Время работы системы: {stats['process_time']} мин")
print(f"Время простоя системы: {stats['downtime']} мин")
print(f"Среднее время ожидания в очереди: {stats['queue_time']} мин")
print(f"Среднее время ожидания в системе: {stats['system_time']} мин")
print(f"Среднее число составов в очереди: {stats['avg_queue_size']}")
print(f"Среднее число составов в системе: {stats['avg_system_size']}")

Нагрузка системы: 0.6742386957118616
Вероятность простоя системы: 0.32576130428813843
Время работы системы: 13484.773914237232 мин
Время простоя системы: 6515.226085762768 мин
Среднее время ожидания в очереди: 41.86480832540536 мин
Среднее время ожидания в системе: 62.09196919676121 мин
Среднее число составов в очереди: 1.3954936108468454
Среднее число составов в системе: 2.069732306558707
